This workbook is intended to overlay colored outlines over images in order to illustrate types of mistakes the MWT makes 

This notebook was used to generate mistakes used in Figure 1.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import six
import os
import sys
sys.path.append('..'); import pathcustomize, about
about.about()


import pathcustomize


import itertools
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
import matplotlib.image as mpimg
import networkx as nx
#from IPython.core.display import Image as display_image

#os.environ.setdefault('MULTIWORM_SETTINGS', 'multiworm_settings')

import waldo.wio as wio
import multiworm
#import collider
#from waldo.collider.viz import show_before_and_after

In [ ]:
# load all experiments from which we will draw examples

e_pics = wio.Experiment(experiment_id='20130702_135704')
e_gaps = wio.Experiment(experiment_id='20130318_131111')
e_spurious = wio.Experiment(experiment_id='20130614_120518')

In [ ]:
# load screen results for the experiment with lots of pictures

ex_id = '20130702_135704'
screen = pd.read_csv('../../data/prep/collision_validate.csv')
screen = screen[screen['eid'] == ex_id]
assert len(screen), "No data for experiment ID"
#print screen.head()
partials = list(screen[screen['ans'] == 5]['bid'])
singles = list(screen[screen['ans'] == 10]['bid'])
doubles = list(screen[screen['ans'] == 20]['bid'])
triples = list(screen[screen['ans'] == 30]['bid'])

In [ ]:
def plot_single_example(bid, experiment, size=(100, 100), center=None, ax=None):

    
    # get outline
    time, o = find_mid_outline(bid, experiment)
    x, y = zip(*o)
    bbox = min(x), min(y), max(x), max(y)
    xmin, ymin, xmax, ymax = min(x), min(y), max(x), max(y)    
    
    # get image
    #time = experiment[bid]['died_t']
    print 'time:', time
    image_time = find_nearest(list(six.iterkeys(experiment.image_files)), time)
    nearest_img = experiment.image_files[image_time]
    im = mpimg.imread(str(nearest_img))

    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(x, y, color='red', lw=2)
    #ax.fill(x, y, alpha=0.05)

    if center is None:
        xmid = (xmin + xmax)/2.0
        ymid = (ymin + ymax)/2.0
    else:
        xmid, ymid = center
    print 'center:', xmid, ymid
    xmin = int(xmid - size[0]/2.0)
    xmax = int(xmid + size[0]/2.0)
    ymin = int(ymid - size[0]/2.0)
    ymax = int(ymid + size[0]/2.0)
    
    
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])   
    ax.set_autoscale_on(False)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.imshow(im.T, cmap=plt.cm.Greys_r)


In [ ]:
def find_closest_outline(nodes, experiment, closest_frame=None, use_first=True):
    outlines = []
    if use_first:
        it_step = 1
    else:
        it_step = -1
    
    for node in nodes:
        i = 0
        while True:
            try:
                outline = multiworm.readers.blob.decode_outline_line(experiment[node], i)
                outlines.append(outline)
                break
            except ValueError:
                pass
            i += it_step
    #print outlines
    return outlines

def find_mid_outline(node, experiment):
    outlines = []
    frames = []
    times = []
    
    bframes, btimes = zip(*[(int(l.split()[0]), float(l.split()[1])) for l in experiment._blob_lines(node)])
    
    # first just try to grab mid outline
    mid= int(len(btimes)/2)
    try:
        outline = multiworm.readers.blob.decode_outline_line(experiment[node], mid)
        time = btimes[mid]
        return time, outline
    except ValueError:
        pass
    
    # if that didn't work grab as many outlines as possible and return mid, working outline
    #print btimes
    for i in range(len(btimes)):    
        try:
            outline = multiworm.readers.blob.decode_outline_line(experiment[node], i)
            outlines.append(outline)
            times.append(btimes[i])
            frames.append(bframes[i])
        except ValueError:
            pass
    mid= len(times)/2
    
    return times[mid], outlines[mid]

#find_mid_outline(node=513, experiment=e_pics)

In [ ]:
def find_nearest_index(seq, value):
    """Return the index of the value in the sequence that is closest to the given value"""
    return (np.abs(np.array(seq)-value)).argmin()

def find_nearest(seq, value):
    """Return the value in the sequence that is closest to the given value"""
    return seq[find_nearest_index(seq, value)]

def frame_time(experiment, frame):
    """Return the time (in seconds) for the given frame from experiment"""
    return experiment.frame_times[int(frame) - 1]

def nearest_image(experiment, frame=None, time=None):
    if frame is None and time is None:
        raise ValueError("either the 'time' or 'frame' keyword argument must be provided")
    if time is None:
        time = frame_time(experiment, frame)
    
    image_time = find_nearest(list(six.iterkeys(experiment.image_files)), time)
    
    return experiment.image_files[image_time]


In [ ]:
def plot_spurious(bid, experiment, size=(100, 100), center=None, ax=None):

    time = experiment[bid]['died_t']
    # get outline
    o = find_closest_outline([bid], experiment=experiment)
    x, y = zip(*o[0])
    bbox = min(x), min(y), max(x), max(y)
    xmin, ymin, xmax, ymax = min(x), min(y), max(x), max(y)    
    
    # get image

    print 'time:', time
    image_time = find_nearest(list(six.iterkeys(experiment.image_files)), time)
    nearest_img = experiment.image_files[image_time]
    im = mpimg.imread(str(nearest_img))

    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(x, y, color='red', lw=2)
    #ax.fill(x, y, alpha=0.05)

    if center is None:
        xmid = (xmin + xmax)/2.0
        ymid = (ymin + ymax)/2.0
    else:
        xmid, ymid = center
    print 'center:', xmid, ymid
    xmin = int(xmid - size[0]/2.0)
    xmax = int(xmid + size[0]/2.0)
    ymin = int(ymid - size[0]/2.0)
    ymax = int(ymid + size[0]/2.0)
    
    
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])   
    ax.set_autoscale_on(False)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.imshow(im.T, cmap=plt.cm.Greys_r)

def plot_missing(time, center, experiment, size=(100,100), ax=None):
    image_time = find_nearest(list(six.iterkeys(experiment.image_files)), time)
    nearest_img = experiment.image_files[image_time]
    im = mpimg.imread(str(nearest_img))

    if ax is None:
        fig, ax = plt.subplots()


    xmid, ymid = center
    print 'center:', xmid, ymid
    xmin = int(xmid - size[0]/2.0)
    xmax = int(xmid + size[0]/2.0)
    ymin = int(ymid - size[0]/2.0)
    ymax = int(ymid + size[0]/2.0)
    
    
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])   
    ax.set_autoscale_on(False)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.imshow(im.T, cmap=plt.cm.Greys_r)
#plot_spurious(bid=195221, experiment=e_spurious)


In [ ]:
fig, ax = plt.subplots(1,6)
fig.set_size_inches(30,10)

# single
plot_single_example(bid=73, experiment=e_pics, ax=ax[0])
# partial
plot_single_example(bid=513, experiment=e_pics, center=(1310, 1288), ax=ax[1])
#double
plot_single_example(bid=34556, experiment=e_pics, ax=ax[2])
#triple
plot_single_example(bid=60440, experiment=e_pics, ax=ax[3])
#missing
time=1892.0 
center= (574.1369863013698, 1052.0730593607307)
plot_missing(time=time, center=center, experiment=e_gaps, ax=ax[4])
#spurious
plot_spurious(bid=195221, experiment=e_spurious, ax=ax[5])


name = 'blob-taxonomy.png'
plt.savefig(name)

### spurious

In [ ]:
e = e_spurious
#e = e_pics
matches = e.prepdata.load('matches')
#print matches.head()
m = matches[matches['good'] == False]
m = m[m['roi'] == True]
spurious = list(m['bid'])
print len(spurious), 'spurious blobs found'

In [ ]:
random_index = np.random.randint(0, len(spurious), 10)
r = np.array(spurious)[random_index]
print r
for bid in r[:]:
    print bid
    plot_spurious(bid, experiment=e)
    plt.show()

In [ ]:
plot_single_example(bid=44965, experiment=e)


### missing

In [ ]:
plot_missing(time=120, center=(15.383648,865.433962), experiment=e_gaps)

In [ ]:
e = e_gaps
#e = e_pics
missing = e.prepdata.load('missing', index_col=0)
random_index = np.random.randint(0, len(missing), 5)
random_missing = missing.iloc[random_index]
for i, r in random_missing.iterrows():
    #print r
    time = float(r['t'])
    center = float(r['x']), float(r['y'])
    print 'time', time, 'center:', center
    plot_missing(time=time, center=center, experiment=e)
    plt.show()

### partial worms

In [ ]:
print 'partial'
plot_single_example(bid=513, experiment=e_pics, center=(1310, 1288))

In [ ]:
random_index = np.random.randint(0, len(partials), 5)
r = np.array(partials)[random_index]
print r
for bid in r:
    print bid
    plot_single_example(bid, experiment=e_pics)
    plt.show()

In [ ]:
notable_partials =[29199, 47999]

### single worms

In [ ]:
print len(singles), 'known single expamples'
print singles[:20]
boring = [72]

In [ ]:
random_index = np.random.randint(0, len(singles), 5)
r = np.array(singles)[random_index]
print r
for bid in r:
    print bid
    plot_single_example(bid, experiment=e_pics)
    plt.show()

In [ ]:
print 'single'
plot_single_example(bid=73, experiment=e_pics)

In [ ]:
bid = 48560
plot_single_example(bid, experiment=e_pics)

In [ ]:
print 'single'
plot_single_example(bid=1292, experiment=e_pics)

### double

In [ ]:
random_index = np.random.randint(0, len(doubles), 5)
r = np.array(doubles)[random_index]
print r
for bid in r:
    print bid
    plot_single_example(bid, experiment=e_pics)
    plt.show()

In [ ]:
notable_doubles = [34556, 17572]

In [ ]:
plot_single_example(bid=535, experiment=e_pics)

In [ ]:
bid=6708
plot_single_example(bid, experiment=e_pics)

### triple

In [ ]:
print len(triples), 'known triple expamples'
print triples[:20]

In [ ]:
random_index = np.random.randint(0, len(triples), 5)
r = np.array(triples)[random_index]
print r
for bid in r:
    print bid
    plot_single_example(bid, experiment=e_pics)
    plt.show()

In [ ]:
notable_tripples = [60440, 45297]

# multi-frame plots

In [ ]:

#ex_id = '20130318_131111'

ex_id = '20130702_135704' # many pics

experiment = wio.Experiment(experiment_id=ex_id)
digraph = experiment.graph.copy()

In [ ]:
#%matplotlib tk
target = 513
f, axs = show_before_and_after(experiment, target=target)
for ax in axs:
    ax.set_autoscale_on(False)
    still = mpimg.imread(str(nearest_image(experiment, frame=experiment[target]['died_f'])))
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.imshow(still.T, cmap=plt.cm.Greys_r)

In [ ]:
def find_outlines(target, digraph, experiment):
    parents, now, children = find_immediate_relatives(target, digraph)
    #children = list(digraph.successors(target))
    #parents = list(digraph.predecessors(target))    
    #now = list(itertools.chain.from_iterable(digraph.predecessors_iter(c) for c in children))
    #now.extend(list(itertools.chain.from_iterable(digraph.successors_iter(p) for p in parents)))
    #now = list(set(now))
    #return parents, children
    print 'c:', children
    print 'now:', now
    print 'p:', parents

    #im_matches = experiment.prepdata.load('matches')
    #im_matches.set_index('bid', inplace=True)
    #all_frames = list(set(im_matches['frame']))
    #print len(all_frames), 'frames'

    parent_outlines = find_closest_outline(parents, use_first=False)
    now_outlines = find_closest_outline(now, use_first=False)
    child_outlines = find_closest_outline(children, use_first=True)

    frame = experiment[target]['died_f']
    pframe = experiment[parents[0]]['died_f']
    cframe = experiment[children[0]]['born_f']

    print pframe, frame, cframe
    frames = [pframe, frame, cframe]
    return frames, parent_outlines, now_outlines, child_outlines

In [ ]:
def find_immediate_relatives(target, digraph):
    children = list(digraph.successors(target))
    parents = list(digraph.predecessors(target))    
    now = list(itertools.chain.from_iterable(digraph.predecessors_iter(c) for c in children))
    now.extend(list(itertools.chain.from_iterable(digraph.successors_iter(p) for p in parents)))
    now = list(set(now))
    return parents, now, children
    

In [ ]:
def plot_outlines(parent_outlines, now_outlines, child_outlines, images, size=100):

    fig = plt.figure()
    fig.set_size_inches(30,10)
    
    gs = gridspec.GridSpec(1, 3)
    ax1 = plt.subplot(gs[0, 0])
    ax2 = plt.subplot(gs[0, 1])
    ax3 = plt.subplot(gs[0, 2])
    axs = [ax1, ax2, ax3]

    bbox = (100000, 100000, 0, 0)
    def compare_bbox(x, y, bbox):
        xmin, ymin, xmax, ymax = bbox
        if min(x) < xmin:
            xmin = min(x)
        if min(y) < ymin:
            ymin = min(y)
        if max(x) > xmax:
            xmax = max(x)
        if max(y) > ymax:
            ymax = max(y)
        return (xmin, ymin, xmax, ymax)

    color_cycle = ax1._get_lines.color_cycle        

    def draw_outlines_on_ax(ax, outlines, bbox):
        for o in outlines:
            x, y = zip(*o)
            #ax.plot(x, y, color='black', lw=0.5)
            #ax.fill(x, y, alpha=0.05, color=color_cycle.next())
            ax.plot(x, y, lw=2, color=color_cycle.next())   
            bbox = compare_bbox(x, y, bbox)
        return bbox

    bbox = draw_outlines_on_ax(axs[0], parent_outlines, bbox)
    bbox = draw_outlines_on_ax(axs[1], now_outlines, bbox)
    bbox = draw_outlines_on_ax(axs[2], child_outlines, bbox)

    # create box around midpoint
    xmin, ymin, xmax, ymax = bbox
    xmid = (xmax + xmin) / 2 
    ymid = (ymax + ymin) / 2
    print bbox
    xmin = int(xmid - size / 2)
    xmax = int(xmid + size / 2)
    ymin = int(ymid - size / 2)
    ymax = int(ymid + size / 2)    
    print (xmin, ymin, xmax, ymax)
 
    for ax, im in zip(axs, images):
        ax.set_xlim([xmin, xmax])
        ax.set_ylim([ymin, ymax])
        ax.set_autoscale_on(False)
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        ax.imshow(im.T, cmap=plt.cm.Greys_r)
    return fig

def plot_outlines_margin(parent_outlines, now_outlines, child_outlines, images, margin=3):

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 3)
    ax1 = plt.subplot(gs[0, 0])
    ax2 = plt.subplot(gs[0, 1])
    ax3 = plt.subplot(gs[0, 2])
    axs = [ax1, ax2, ax3]

    bbox = (100000, 100000, 0, 0)
    def compare_bbox(x, y, bbox):
        xmin, ymin, xmax, ymax = bbox
        if min(x) < xmin:
            xmin = min(x)
        if min(y) < ymin:
            ymin = min(y)
        if max(x) > xmax:
            xmax = max(x)
        if max(y) > ymax:
            ymax = max(y)
        return (xmin, ymin, xmax, ymax)

    color_cycle = ax1._get_lines.color_cycle        

    def draw_outlines_on_ax(ax, outlines, bbox):
        for o in outlines:
            x, y = zip(*o)
            #ax.plot(x, y, color='black', lw=0.5)
            #ax.fill(x, y, alpha=0.05, color=color_cycle.next())
            ax.plot(x, y, lw=2, color=color_cycle.next())   
            bbox = compare_bbox(x, y, bbox)
        return bbox


    bbox = draw_outlines_on_ax(axs[0], parent_outlines, bbox)
    bbox = draw_outlines_on_ax(axs[1], now_outlines, bbox)
    bbox = draw_outlines_on_ax(axs[2], child_outlines, bbox)

    xmin, ymin, xmax, ymax = bbox

    for ax, im in zip(axs, images):
        ax.set_xlim([xmin-margin, xmax+margin])
        ax.set_ylim([ymin-margin, ymax+margin])
        ax.set_autoscale_on(False)
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        ax.imshow(im.T, cmap=plt.cm.Greys_r)
    return fig

In [ ]:
def plot_target(target, digraph, experiment, margin=False):
    frames, parent_outlines, now_outlines, child_outlines = find_outlines(target, digraph, experiment)
    images = [mpimg.imread(str(nearest_image(experiment, frame=f))) for f in frames]

    fig = plot_outlines(parent_outlines, now_outlines, child_outlines, images)
    if margin:
        fig = plot_outlines_margin(parent_outlines, now_outlines, child_outlines, images)

In [ ]:
target=513

plot_target(target, digraph, experiment)
plt.savefig('{t}-split.png'.format(t=target))

In [ ]:
random_index = np.random.randint(0, len(partials), 10)
r = np.array(partials)[random_index]
print r
for target in r:
    try:
        print bid
        plot_target(target, digraph, experiment)
        name = '{t}-collision.png'.format(t=target)
        #print name
        #plt.savefig(name)
        plt.show()
    except:
        print bid, 'failed'

In [ ]:
notable_collisions = [15081, 78167, 16500, 28028]

In [ ]:
random_index = np.random.randint(0, len(doubles), 10)
r = np.array(doubles)[random_index]
print r
for target in notable_collisions:
    try:
        print bid
        plot_target(target, digraph, experiment)
        name = '{t}-collision.png'.format(t=target)
        print name
        plt.show()
        #plt.savefig(name)
    except:
        print bid, 'failed'

In [ ]:
canidates = []
for i, node in enumerate(digraph):
    parents, now, children = find_immediate_relatives(node, digraph)
    
    all_good = True
    #find_closest_outline([node], use_first=True)

    try:
        find_closest_outline([node],  use_first=True)
    except:
        all_good = False
        continue
    
    relatives = []
    relatives.extend(parents)
    relatives.extend(now)
    relatives.extend(children)
    relatives = list(set(relatives))
    
    try:
        for n in relatives:
            experiment[n]
        find_closest_outline(relatives,  use_first=True)
    except:
        all_good = False
        continue
    #print children, parents, all_good
    if children and parents and all_good:
        canidates.append(node)
    if i > 1000:
        break
print len(canidates), 'canidates found'

In [ ]:
target=canidates[4]
plot_target(target, digraph, experiment, margin=True)


In [ ]:
target=canidates[8]
plot_target(target, digraph, experiment, margin=True)


In [ ]:
target=canidates[9]
plot_target(target, digraph, experiment, margin=False)


In [ ]:
target=canidates[10]
plot_target(target, digraph, experiment, margin=False)


In [ ]:
target=canidates[15]
plot_target(target, digraph, experiment, margin=False)


In [ ]:
target=canidates[40]
plot_target(target, digraph, experiment, margin=False)
plt.savefig('{t}-example.png'.format(t=target))


In [ ]:
target=canidates[50]
plot_target(target, digraph, experiment, margin=False)
plt.savefig('{t}-example.png'.format(t=target))

In [ ]:
target=canidates[160]
plot_target(target, digraph, experiment, margin=False)
plt.savefig('{t}-example.png'.format(t=target))

In [ ]:
target=canidates[162]
plot_target(target, digraph, experiment, margin=False)
plt.savefig('{t}-example.png'.format(t=target))

In [ ]:
target=canidates[135]
plot_target(target, digraph, experiment, margin=False)
plt.savefig('{t}-example.png'.format(t=target))

In [ ]:
target=canidates[124]
plot_target(target, digraph, experiment, margin=False)

plt.savefig('{t}-example.png'.format(t=target))

In [ ]:
target=canidates[122]

plot_target(target, digraph, experiment, margin=False)
plt.savefig('{t}-example.png'.format(t=target))

In [ ]:
target=canidates[44]
plot_target(target, digraph, experiment, margin=False)
